# Анализ базы данных рейтингов книг

## Оглавление
* [Цель исследования](#purpose)
* [Исходные данные](#data)
* [Количество книг, которые вышли после 01.01.2000](#books)
* [Среднее количество обзоров и средняя оценка для каждой книги](#av_rev_rate)
* [Издательство, которое выпустило больше всего книг](#pub)
* [Автор с самой высокой средней оценкой книг](#author)
* [Среднее количество обзоров от активных пользователей](#users)
* [Выводы](#conclusions)


## Цель исследования <a id="purpose"></a>

От заказчика поступило ТЗ по анализу базы данных рейтингов книг:

- Посчитайте, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитайте количество обзоров и среднюю оценку;
- Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
- Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
- Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

## Исходные данные <a id="data"></a>

In [ ]:
# импортируем библиотеки
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

In [ ]:
# устанавливаем параметры соединения к БД
db_config = {'user': 'praktikum_student', # имя пользователя
 'pwd': 'Sdf4$2;d-d30pp', # пароль
 'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
 'port': 6432, # порт подключения
 'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

In [ ]:
def select(sql):
  return pd.io.sql.read_sql(sql, con = engine)

In [ ]:
sql = '''SELECT * FROM books
LIMIT 5
'''

In [ ]:
select(sql)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [ ]:
sql = '''SELECT * FROM authors
LIMIT 5
'''

In [ ]:
select(sql)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [ ]:
sql = '''SELECT * FROM ratings
LIMIT 5
'''

In [ ]:
select(sql)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [ ]:
sql = '''SELECT * FROM reviews
LIMIT 5
'''

In [ ]:
select(sql)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [ ]:
sql = '''SELECT * FROM publishers
LIMIT 5
'''

In [ ]:
select(sql)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


## Количество книг, которые вышли после 01.01.2000 <a id="books"></a>

Найдем, сколько книг вышло после 1 января 2000 года.

In [ ]:
sql = '''
SELECT COUNT(DISTINCT book_id) AS cnt
FROM books
WHERE publication_date > '2000-01-01'
'''

In [ ]:
select(sql)

,cnt
0,819


После 01.01.2000 вышло **819** книг (подсчет по уникальным идентификаторам книг в системе).

In [ ]:
sql = '''
SELECT title, COUNT(book_id) as cnt_book_id
FROM books
GROUP BY title
HAVING COUNT(book_id) > 1

'''

In [ ]:
select(sql)

,title,cnt_book_id
0,Memoirs of a Geisha,2


In [ ]:
sql = '''
SELECT *
FROM books
WHERE title = 'Memoirs of a Geisha'

'''

In [ ]:
select(sql)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,426,39,Memoirs of a Geisha,434,2005-11-15,241
1,427,39,Memoirs of a Geisha,503,2005-11-22,311


Вычислили книгу с разными идентификаторами, но с одним названием. У нее разное количество страниц, отличается дата издания и издательство. Будем считать, что это разные книги. Возможно, разные переиздания. Укажем это отдельно в выводах.

## Среднее количество обзоров и средняя оценка для каждой книги <a id="av_rev_rate"></a>

Для каждой книги посчитаем количество обзоров и среднюю оценку.

In [ ]:
sql = '''
SELECT books.title, AVG(rating) AS avg_rating, COUNT(DISTINCT review_id) AS cnt_review
FROM books
LEFT JOIN ratings on ratings.book_id = books.book_id
LEFT JOIN reviews on reviews.book_id = books.book_id
GROUP BY books.book_id
LIMIT 3
'''

In [ ]:
select(sql)

,title,avg_rating,cnt_review
0,'Salem's Lot,3.666667,2
1,1 000 Places to See Before You Die,2.500000,1
2,13 Little Blue Envelopes (Little Blue Envelope...,4.666667,3


Вывели средние оценки и количество уникальных отзывов для каждой книги.

In [ ]:
select(sql = '''
SELECT books.title, AVG(rating) AS avg_rating, COUNT(DISTINCT review_id) AS cnt_review
FROM books
LEFT JOIN ratings on ratings.book_id = books.book_id
LEFT JOIN reviews on reviews.book_id = books.book_id
GROUP BY books.book_id
LIMIT 3
''')

,title,avg_rating,cnt_review
0,'Salem's Lot,3.666667,2
1,1 000 Places to See Before You Die,2.500000,1
2,13 Little Blue Envelopes (Little Blue Envelope...,4.666667,3


## Издательство, которое выпустило больше всего книг <a id="pub"></a>

Определим издательство, которое выпустило наибольшее число книг толще 50 страниц.

In [ ]:
sql = '''
SELECT publisher, COUNT(book_id) AS cnt_books
FROM books
LEFT JOIN publishers on publishers.publisher_id = books.publisher_id
WHERE books.num_pages > 50
GROUP BY publisher
ORDER BY cnt_books DESC
LIMIT 1
'''

In [ ]:
select(sql)

,publisher,cnt_books
0,Penguin Books,42


Издательство **Penguin Books** выпустило 42 книги - это наибольший показатель среди всех рассматриваемых издательств.

## Автор с самой высокой средней оценкой книг <a id="author"></a>

Определим автора с самой высокой средней оценкой книг — учитываем только книги с 50 и более оценками.

In [ ]:
sql = '''
WITH high_rating_books AS (
SELECT books.book_id, COUNT(rating) AS cnt_rating, AVG(rating) AS avg_rating
FROM books
LEFT JOIN ratings on ratings.book_id = books.book_id
GROUP BY books.book_id
HAVING COUNT(rating) >= 50
ORDER BY cnt_rating DESC)

SELECT author, AVG(avg_rating) as avg_author_rating
FROM books
RIGHT JOIN high_rating_books on high_rating_books.book_id = books.book_id
LEFT JOIN authors on authors.author_id = books.author_id
GROUP BY author
ORDER BY avg_author_rating DESC
LIMIT 1
'''

In [ ]:
select(sql)

,author,avg_author_rating
0,J.K. Rowling/Mary GrandPré,4.283844


Автор с самой высокой оценкой книг (среди книг с 50 и более оценками) - **J.K. Rowling/Mary GrandPré.**

##  Среднее количество обзоров от активных пользователей <a id="users"></a>

Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [ ]:
sql = '''
WITH active_users_review AS (
WITH active_users AS (
SELECT ratings.username, COUNT(rating_id) AS cnt_rating
FROM ratings
GROUP BY ratings.username
HAVING COUNT(rating_id) > 50
ORDER BY cnt_rating DESC)

SELECT reviews.username, COUNT(review_id) as cnt_reviews_from_active_users
FROM reviews
RIGHT JOIN active_users on reviews.username = active_users.username
GROUP BY reviews.username)

SELECT AVG(active_users_review.cnt_reviews_from_active_users)
FROM active_users_review
'''

In [ ]:
select(sql)

,avg
0,24.333333


Среднее количество обзоров от самых активных пользователей (более 50 оценок) - **24,3**. 

## Выводы  <a id="conclusions"></a>

Мы выполнили исследование базы данных книг и определили:

* После 01.01.2000 вышло **819** книг (подсчет по уникальным идентификаторам книг в системе);
* Вывели средние оценки для 1000 уникальных идентификаторов книг и количество отзывов для 994 книг (для 6 отзывы отсутствуют);
* Издательство **Penguin Books** выпустило 42 книги, это максимум среди конкурентов;
* Автор с самой высокой оценкой книг (среди книг с 50 и более оценками) - **J.K. Rowling/Mary GrandPré**;
* Среднее количество обзоров от самых активных пользователей (более 50 оценок) - **24,3**. 

*Примечание: В таблице `books` обнаружили аномалию: книга с одним названием фигурирует под двумя разными идентификаторами. Однако в каждом из вариантов разное количество страниц. Даты издания и название издательства также отличаются. Оставили без изменения эти позиции.*
